In [1]:
import os
import torch
import numpy as np
import cv2 as cv
import pandas as pd
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
sys.path.insert(1, '/home/kseuro/Kai/deeplearnphysics/pytorch/particle_generator/')

# My stuff
import ae
import utils
from ewm import ewm_G

In [3]:
# Set the root path of the AutoEncoder model experiments
AE_root = "/media/hdd1/kai/particle_generator/experiments/larcv_ae/"

In [4]:
# Set the root path of the EWM trained Generator model experiments
EWM_root = "/media/hdd1/kai/particle_generator/experiments/ewm_models/"

In [5]:
# Path to model weights
weights_dir = "weights/"

In [6]:
# Set the GPU (GPU 1 is the best option)
device = torch.device(1)

# Select the AutoEncoder and Generator Models you want to deploy

## Get the names of all the saved AutoEncoder experiments in the exp_root folder

In [7]:
AE_paths = []
for path in os.listdir(AE_root):
    AE_paths.append(os.path.join(AE_root, path))

print("-"*60)
for i in range(len(AE_paths)):
    print("\n Exp_{}:".format(str(i)), AE_paths[i], '\n')
    print("-"*60)

------------------------------------------------------------

 Exp_0: /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_256 

------------------------------------------------------------

 Exp_1: /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20_15000-epochs 

------------------------------------------------------------

 Exp_2: /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20 

------------------------------------------------------------

 Exp_3: /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_256_1000-epochs 

------------------------------------------------------------

 Exp_4: /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_128_20 

------------------------------------------------------------


## Select the AutoEncoder experiment you want

In [8]:
AE_dir = AE_paths[2]

In [9]:
# Create the full path to the AutoEncoder experiment
AE_path = os.path.join(AE_root, AE_dir) + "/"
print("Path to AutoEncoder set as: \n{}".format(AE_path))

Path to AutoEncoder set as: 
/media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20/


## Get the names of all the saved EWM Generator models

In [10]:
EWM_paths = []
for path in os.listdir(EWM_root):
    EWM_paths.append(os.path.join(EWM_root, path))

print("-"*60)
for i in range(len(EWM_paths)):
    print("\n Exp_{}:".format(str(i)), EWM_paths[i], '\n')
    print("-"*60)

------------------------------------------------------------

 Exp_0: /media/hdd1/kai/particle_generator/experiments/ewm_models/ewm_test_model 

------------------------------------------------------------


## Select the EWM model experiment you want

In [11]:
EWM_dir = EWM_paths[0]

In [12]:
# Create the full path to the EWM model
EWM_path = os.path.join(EWM_root, EWM_dir) + "/"
print("Path to EWM Generator Model set as: \n{}".format(EWM_path))

Path to EWM Generator Model set as: 
/media/hdd1/kai/particle_generator/experiments/ewm_models/ewm_test_model/


# Load selected models from checkpoint and onto GPU

## Load the AE config .csv as a dict
- Get the model architecture from the config_df

In [13]:
config_csv = AE_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

In [14]:
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].item())**2
im_dim   = int(np.sqrt(im_size))

## Set up AutoEncoder model on the GPU

In [15]:
# Set up AE layer sizes
base = [256] 

# Compute encoder sizes
sizes = lambda: [ (yield 2**i) for i in range(n_layers) ]
enc_sizes = base * n_layers
enc_sizes = [a*b for a,b in zip(enc_sizes, [*sizes()])][::-1]

# Update kwarg dicts
# Decoder is the reverse of the encoder
ae_kwargs = {'enc_sizes' : enc_sizes, 'l_dim' : l_dim, 'im_size' : im_size, 'dec_sizes' : enc_sizes[::-1]}

In [16]:
AE = ae.AutoEncoder(**ae_kwargs).to(device)

## Load the AE model checkpoint

In [17]:
# Get checkpoint name(s)
AE_checkpoint_path  = AE_path + weights_dir
AE_checkpoint_names = []
for file in os.listdir(AE_checkpoint_path):
    AE_checkpoint_names.append(os.path.join(AE_checkpoint_path, file))

In [18]:
print("-"*60)
for i in range(len(AE_checkpoint_names)):
    print("\n Chkpt_{} :".format(str(i)), '\n', AE_checkpoint_names[i], '\n')
    print("-"*60)

------------------------------------------------------------

 Chkpt_0 : 
 /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20/weights/best_ae_ep_7000.tar 

------------------------------------------------------------

 Chkpt_1 : 
 /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20/weights/best_ae_ep_7499.tar 

------------------------------------------------------------

 Chkpt_2 : 
 /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20/weights/best_ae_ep_5250.tar 

------------------------------------------------------------


In [19]:
# Open the checkpoint analysis file in the experiment folder
checkpoint_evaluation = AE_path + "checkpoint_evaluation.txt"
with open(checkpoint_evaluation, 'r') as f_obj:
    print(f_obj.readlines())

FileNotFoundError: [Errno 2] No such file or directory: '/media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20/checkpoint_evaluation.txt'

In [20]:
# Select the desired checkpoint from the list
AE_checkpoint = AE_checkpoint_names[0]

In [21]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(AE_checkpoint)

In [22]:
# Load the model's state dictionary
# Note: The IncompatibleKeys(missing_keys=[], unexpected_keys=[]) message indicates that
#       there were no problems in loading the state dictionary. Bit confusing...
AE.load_state_dict(checkpoint['state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [23]:
# Put the model in evaluation mode
AE.eval()

AutoEncoder(
  (encoder): Encoder(
    (fc_blocks): Sequential(
      (0): Sequential(
        (0): Linear(in_features=4096, out_features=2048, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (1): Sequential(
        (0): Linear(in_features=2048, out_features=1024, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (2): Sequential(
        (0): Linear(in_features=1024, out_features=512, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (3): Sequential(
        (0): Linear(in_features=512, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
    )
    (last): Linear(in_features=256, out_features=20, bias=True)
  )
  (decoder): Decoder(
    (fc_blocks): Sequential(
      (0): Sequential(
        (0): Linear(in_features=20, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): Leaky

## Load the G config .csv as a dict
- Get the model architecture from the config_df

In [24]:
config_csv = EWM_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

In [25]:
# Get the model architecture from config df
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].item())
n_hidden = int(config_df[config_df['Unnamed: 0'].str.contains("n_hidden")==True]['0'].item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].item())
z_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("z_dim")==True]['0'].item())
print("{} Layer model with {} hidden units per layer".format(n_layers, n_hidden))
print("Mapping {}_dim_Gaussian to {}_dim code vectors".format(z_dim, l_dim))

4 Layer model with 512 hidden units per layer
Mapping 100_dim_Gaussian to 20_dim code vectors


## Set up the Generator model

In [26]:
# Model kwargs
fc_sizes = [n_hidden] * n_layers
ewm_kwargs = { 'z_dim':z_dim, 'fc_sizes':fc_sizes, 'n_out':l_dim }

In [27]:
# Create generator on GPU
G = ewm_G(**ewm_kwargs).to(device)
G.weights_init()

## Load EWM model checkpoint

In [28]:
# Get checkpoint name(s)
EWM_checkpoint_path  = EWM_path + weights_dir
EWM_checkpoint_names = []
for file in os.listdir(EWM_checkpoint_path):
    EWM_checkpoint_names.append(os.path.join(EWM_checkpoint_path, file))

In [29]:
print("-"*60)
for i in range(len(EWM_checkpoint_names)):
    print("\n {} :".format(str(i)), EWM_checkpoint_names[i], '\n')
    print("-"*60)

------------------------------------------------------------

 0 : /media/hdd1/kai/particle_generator/experiments/ewm_models/ewm_test_model/weights/best_ewm_ep_99.tar 

------------------------------------------------------------

 1 : /media/hdd1/kai/particle_generator/experiments/ewm_models/ewm_test_model/weights/best_ewm_ep_0.tar 

------------------------------------------------------------

 2 : /media/hdd1/kai/particle_generator/experiments/ewm_models/ewm_test_model/weights/best_ewm_ep_1.tar 

------------------------------------------------------------


In [30]:
# Select the checkpoint you want
EWM_checkpoint = EWM_checkpoint_names[0]

In [31]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(EWM_checkpoint)

In [32]:
# Load the model's state dictionary
# Note: The IncompatibleKeys(missing_keys=[], unexpected_keys=[]) message indicates that
#       there were no problems in loading the state dictionary. Bit confusing...
G.load_state_dict(checkpoint['state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [33]:
# Put the model in evaluation mode
G.eval()

ewm_G(
  (fc): Sequential(
    (0): Sequential(
      (0): Linear(in_features=100, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (1): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (2): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
  )
  (out): Sequential(
    (0): Linear(in_features=512, out_features=20, bias=True)
  )
)

## Check that the l_dim of the AE is the same as the out_dim of the Generator

In [34]:
ae_l_dim = AE.encoder.last.out_features
g_out_dim = G.out[0].out_features

In [35]:
if ae_l_dim != g_out_dim:
    print("Code vector sizes do not match!")
    print("AE: {} | G: {}".format(ae_l_dim, g_out_dim))
else:
    print("Code vector sizes match")

Code vector sizes match


## Deploy the Decoder using G as an input

In [45]:
samples = []; limit = 24

In [46]:
for itr in range(limit):
    # Create an input vector for G
    z_rand = torch.randn(1, z_dim).to(device)
    
    # Get a code vector from G
    code_vec = G(z_rand)
    
    # Move code_vector onto GPU
    code_vec = code_vec.to(device)
    
    # Push code_vec through Decoder
    sample = AE.decoder(code_vec)
    
    # Reshape the sample, detach, and convert to numpy array
    samples.append(sample.view(im_dim, im_dim).detach().cpu().numpy())
    
    if itr +1 == limit:
        break

In [47]:
print(samples[0].shape, type(samples[0]))

(64, 64) <class 'numpy.ndarray'>


## Viz the deploy samples

In [50]:
n_row = 3; n_col = limit // n_row

fig, axes = plt.subplots(n_row, n_col)
supTitle = "{} model deploy samples using {}x{} dataset \n and {}_dim code vectors".format('Decoder', im_dim, im_dim, l_dim)
fig.suptitle(supTitle, color="white")

img = None; test_num = limit; sample_num = limit - 1
for i in range(0, n_row):
    for j in range(0, n_col):
        img = axes[i,j].imshow( samples[sample_num] )
        axes[i,j].spines['bottom'].set_color('white')
        axes[i,j].spines['top'].set_color('white')
        axes[i,j].spines['left'].set_color('white')
        axes[i,j].spines['right'].set_color('white')
        axes[i,j].xaxis.label.set_color('white')
        axes[i,j].yaxis.label.set_color('white')
        axes[i,j].get_xaxis().set_visible(False)
        axes[i,j].get_yaxis().set_visible(False)
        axes[i,j].tick_params(axis='x', colors='#443941')
        axes[i,j].tick_params(axis='y', colors='#443941')
        sample_num -= 1

# Colorbar
# cbar_axes = fig.add_axes([0.9, 0.1, 0.03, 0.8])
# cbar = plt.colorbar(img, cax=cbar_axes)
# cbar.ax.yaxis.set_tick_params(color="white")
# cbar.outline.set_edgecolor(color="white")
# plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color="white")

plt.savefig("{}_model_deploy_samples_{}_dataset_{}_code_vectors.png".format('Decoder', im_dim, l_dim), dpi=300, facecolor='#443941')

0 0 23
0 1 22
0 2 21
0 3 20
0 4 19
0 5 18
0 6 17
0 7 16
1 0 15
1 1 14
1 2 13
1 3 12
1 4 11
1 5 10
1 6 9
1 7 8
2 0 7
2 1 6
2 2 5
2 3 4
2 4 3
2 5 2
2 6 1
2 7 0
